In [14]:
from dave.proxies.data import CrystalFeat
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from pathlib import Path
import torch
import numpy as np

In [15]:
BASE_PATH = Path(".").resolve().parent
DATA_PATH = BASE_PATH.parent / "ic_data" / "nrcc_ionic_conductivity"
print(DATA_PATH)

/home/minion/Downloads/ic_data/nrcc_ionic_conductivity


In [16]:
x_mean = torch.load(str(DATA_PATH / "x.mean"))
x_std = torch.load(str(DATA_PATH / "x.std"))

trainset = CrystalFeat(root=str(DATA_PATH), target="Ionic conductivity (S cm-1)", subset="train", scalex={"mean":x_mean, "std":x_std})
x_train = []
y_train = []

for i, (x, y) in enumerate(trainset):
    a, b, c = x
    # print(a[a != 0])
    # if i == 10:
    #     break
    b = b.unsqueeze(-1)
    x = torch.cat((a, b, c), dim=-1)
    x_train.append(x.numpy())
    y_train.append(y.numpy())
idx = np.arange(len(x_train))
print(max(idx) / 5)
np.random.shuffle(idx)
x_train = np.array(x_train)[idx]
y_train = np.array(y_train)[idx]

86.0


In [25]:
# hparams = {
#     "n_estimators": [50, 100, 150],
#     "max_depth": [12, 24, 36],
#     "max_features": ["log2", "sqrt"],
#     "min_samples_leaf": [2, 3, 4, 5]
# }

hparams = {
    'max_depth': 12,
    'max_features': 'sqrt',
    'min_samples_leaf': 2,
    'n_estimators': 50
}

model = RandomForestRegressor(**hparams)
res = cross_validate(model, x_train, y_train, cv=5, scoring="neg_mean_absolute_error")
print(-np.mean((res["test_score"])))
# gs = GridSearchCV(estimator=model, param_grid=hparams, cv=5, scoring="")
# gs.fit(x_train, y_train)
# best = gs.best_score_
# print(best)
# print(gs.best_estimator_)
# print(gs.best_params_)
# y_train = np.array(y_train).reshape(-1, 1)
# y_pred = model.predict(x_train)


1.3860985837983129


In [33]:
hparams = {
    "hidden_layer_sizes": [128, 128, 128, 128, 128],
    "activation": "relu",
    "solver": "adam",
    "learning_rate_init": 0.003,
    # "validation_fraction" :0.2,
    "batch_size": 128,
    "max_iter": 40,
    "learning_rate": "adaptive"
}

model = MLPRegressor(**hparams)
res = cross_validate(model, x_train, y_train, cv=5, scoring="neg_mean_absolute_error")
print(-np.mean((res["test_score"])))
# gs = GridSearchCV(estimator=model, param_grid=hparams, cv=5)
# model.fit(x_train[-86:], y_train[-86:])
# model.score(x_train[-86:], y_train[-86:])
# best = gs.best_score_
# print(best)
# print(gs.best_estimator_)
# print(gs.best_params_)

/home/minion/anaconda3/envs/materials/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (40) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/minion/anaconda3/envs/materials/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (40) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/minion/anaconda3/envs/materials/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (40) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/minion/anaconda3/envs/materials/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (40) reached and the optimization hasn't converged yet.
 

1.7561132192611695


/home/minion/anaconda3/envs/materials/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (40) reached and the optimization hasn't converged yet.
  warnings.warn(
